In [ ]:
import sqlite3
import pandas as pd
import re
import keras
from unicodedata import normalize
import utils.functions as custom_functions

Using TensorFlow backend.


In [ ]:
import imp
imp.reload(custom_functions)

In [ ]:
sqlite_file = 'data/database.sqlite'
db = custom_functions.create_connection(sqlite_file)

In [ ]:
# get a small sample
small_sample = custom_functions.get_sample(nrows=20000, db=db)
display(small_sample)

# save as csv
small_sample.to_csv('data/reddit_small_sample.csv', index = False)

In [ ]:
# TODO
# get bigger samples, use remove duplicates, and merge

# neet to create a list of ids from sample_df_1
#df_2, ex_id = remove_duplicates(sample_df_2, set(sample_df_1.id.unique()))